In [0]:
%sql
USE CATALOG catproject_catalog

In [0]:
%sql
USE SCHEMA ecom_bronze

In [0]:
%sql
CREATE TABLE ecom_bronze.product_raw (
  product_id          STRING,
  product_name        STRING,
  category            STRING,
  brand               STRING,
  description         STRING,
  price               DECIMAL(18,2),
  cost                DECIMAL(18,2),
  currency            STRING,
  status              STRING,
  created_at          TIMESTAMP,
  updated_at          TIMESTAMP,
  source_system       STRING,
  ingestion_time      TIMESTAMP,
  ingestion_year      INT,
  ingestion_month     INT,
  operation_type      STRING,
  record_hash         STRING,
  source_updated_at   TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month)
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = true,
  'comment' = 'Bảng raw chứa thông tin sản phẩm từ hệ thống nguồn (ERP, CMS, POS)'
);


Tạo Constraint để đảm bảo Data Integrity

In [0]:
%sql
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN product_id SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN product_name SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN category SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN brand SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN price SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN cost SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN currency SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN status SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE ecom_bronze.product_raw ALTER COLUMN created_at SET NOT NULL;


ALTER TABLE ecom_bronze.product_raw ADD CONSTRAINT valid_status CHECK (status IN ('active', 'inactive', 'discontinued'));
ALTER TABLE ecom_bronze.product_raw ADD CONSTRAINT positive_price CHECK (price >= 0);
ALTER TABLE ecom_bronze.product_raw ADD CONSTRAINT positive_cost CHECK (cost >= 0);
ALTER TABLE ecom_bronze.product_raw ADD CONSTRAINT valid_currency CHECK (currency IN ('USD','VND'));
ALTER TABLE ecom_bronze.product_raw ADD CONSTRAINT valid_created_date CHECK (created_at <= current_timestamp());


Tạo comment cho các column

In [0]:
%sql
COMMENT ON COLUMN ecom_bronze.product_raw.product_id IS 'Mã sản phẩm duy nhất';
COMMENT ON COLUMN ecom_bronze.product_raw.product_name IS 'Tên sản phẩm';
COMMENT ON COLUMN ecom_bronze.product_raw.category IS 'Danh mục sản phẩm';
COMMENT ON COLUMN ecom_bronze.product_raw.brand IS 'Thương hiệu sản phẩm';
COMMENT ON COLUMN ecom_bronze.product_raw.description IS 'Mô tả sản phẩm';
COMMENT ON COLUMN ecom_bronze.product_raw.price IS 'Giá bán';
COMMENT ON COLUMN ecom_bronze.product_raw.cost IS 'Giá vốn sản phẩm';
COMMENT ON COLUMN ecom_bronze.product_raw.currency IS 'Loại tiền tệ';
COMMENT ON COLUMN ecom_bronze.product_raw.status IS 'Trạng thái kinh doanh (active/inactive/discontinued)';
COMMENT ON COLUMN ecom_bronze.product_raw.created_at IS 'Thời gian tạo bản ghi ở source';
COMMENT ON COLUMN ecom_bronze.product_raw.updated_at IS 'Thời gian cập nhật gần nhất ở source';
COMMENT ON COLUMN ecom_bronze.product_raw.source_system IS 'Hệ thống nguồn (ERP, CMS, POS)';
COMMENT ON COLUMN ecom_bronze.product_raw.ingestion_time IS 'Thời điểm ingest vào lakehouse';
COMMENT ON COLUMN ecom_bronze.product_raw.ingestion_year IS 'Năm ingest (partition)';
COMMENT ON COLUMN ecom_bronze.product_raw.ingestion_month IS 'Tháng ingest (partition)';
COMMENT ON COLUMN ecom_bronze.product_raw.operation_type IS 'INSERT / UPDATE / DELETE';
COMMENT ON COLUMN ecom_bronze.product_raw.record_hash IS 'Hash để kiểm tra thay đổi';
COMMENT ON COLUMN ecom_bronze.product_raw.source_updated_at IS 'Thời điểm cập nhật dữ liệu tại source';


In [0]:
%sql
DESCRIBE TABLE EXTENDED ecom_bronze.product_raw;